In [48]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import spacy
!python3 -m spacy download en
import en_core_web_sm
ner_model = en_core_web_sm.load()
import pickle
from names_dataset import NameDataset
name_dataset = NameDataset()
import os
import re
org_df = pd.read_csv('/home/jackragless/projects/data/DAIC_GLOGEN/companies_and_unis.csv')
unigram = pd.read_csv('/home/jackragless/projects/data/DAIC_GLOGEN/unigram_freq.csv')
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))  

     |████████████████████████████████| 12.0 MB 1.5 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/jackragless/miniconda3/lib/python3.8/site-packages/en_core_web_sm -->
/home/jackragless/miniconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [49]:
def clean_text(wiki_object):
    
    test_str = wiki_object['text']
    if test_str.find('Version history') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('Version history', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('See also') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('See also', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('References') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('References', i)] 
        test_str = test_str[:result[-1]]
        
        
    clean_text = re.sub("[\[\(].*?[\]\)]", "", test_str.replace('\n',''))
    final = ''
    for i in clean_text:
        if i.isalpha() or i.isdigit() or i=='-' or i==' ' or i=='.':
            final += i
            
    
    for i in range(len(final)-2):
        if (final[i].isalpha() and final[i].islower() and final[i]!=' ') and (final[i+1] == '.') and (final[i+2].isalpha() and final[i+2].isupper()):
            final = final[:i+1] + '. ' + final[i+2:]
            
    final = re.sub(' +', ' ', final).replace(' .','.').strip()
    
    wiki_object['text'] = final
    
    wiki_object['kw'].append(wiki_object['title'])
    
    return wiki_object

In [50]:
def clean_text_keep_grammar(wiki_object):
    
    test_str = wiki_object['text']
    if test_str.find('Version history') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('Version history', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('See also') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('See also', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('References') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('References', i)] 
        test_str = test_str[:result[-1]]
        
        
    final = test_str.replace('\n','').replace('"',"'").replace('=','')
            
    
    for i in range(len(final)-2):
        if (final[i].isalpha() and final[i].islower() and final[i]!=' ') and (final[i+1] == '.') and (final[i+2].isalpha() and final[i+2].isupper()):
            final = final[:i+1] + '. ' + final[i+2:]
            
    final = re.sub(' +', ' ', final).replace(' .','.').strip()
    
    wiki_object['text'] = final
    
    wiki_object['kw'].append(wiki_object['title'])
    
    return wiki_object

In [51]:
# def remove_stopwords(text):
#     text = text['text']
#     final = ''
#     for sent in nltk.sent_tokenize(text):
#         sent = sent[:-1]
#         temp_sent = ''
#         for word in sent.split():
#             if word.lower() not in stop_words:
#                 temp_sent += ' ' + word
#         final += ' ' + temp_sent.strip() + '.'
#     return final.strip()

In [52]:
def consec_cap_detect(word_string):
    for i in range(0,len(word_string)-1):
        if word_string[i].isupper() and word_string[i+1].isupper():
            return True
    return False

In [53]:
def person_detect(candidate_string):
    candidate_string = candidate_string.split()
    if len(candidate_string) == 2:
        if name_dataset.search_first_name(candidate_string[0]) == True and name_dataset.search_last_name(candidate_string[1]) == True:
            return True
    elif len(candidate_string) == 3:
        if name_dataset.search_first_name(candidate_string[0]) == True and len(candidate_string[1])>=2 and candidate_string[1][0].isalpha() and candidate_string[1][1] == '.':
            return True
    return False

In [54]:
common_unigram = list(unigram[:10000]['word'])

def common_word_detect(candidate_string): #could add stopwords
    if candidate_string.lower() in common_unigram:
        return True
    else:
        return False

In [55]:
def location_name_detect(whole_text):
    final = []
    doc = ner_model(whole_text)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            final.append(ent.text)
    return final

In [56]:
def misc_filters(candidate_string):
    if len(candidate_string) == 1:
        return True
    elif not any(c.isalpha() for c in candidate_string):
        return True
    return False

In [57]:
def label_keyword_array(kw_arr, text):
    temp_dict = {}
    for i in kw_arr:
        if consec_cap_detect(i):
            temp_dict[i] = 'P'
        else:
            temp_dict[i] = 'K'
            
    for j in temp_dict:
        if (person_detect(j) or common_word_detect(j) or misc_filters(j)) and temp_dict[j]!='P':
            temp_dict[j] = 'R'
            
    location_ners = location_name_detect(text)
    for k in temp_dict:
        if k in location_ners and temp_dict[j]!='P':
            temp_dict[k] = 'R'
        elif k.lower() in org_df:
            temp_dict[k] = 'R'
        elif len(k.split()) > 4:
            temp_dict[k] = 'R'
        elif text.lower().find(k.lower()) == -1: #find if other reference content introduced via oversight
            temp_dict[k] = 'R'
    
    return temp_dict

In [58]:
with open('/home/jackragless/projects/data/DAIC_GLOGEN/aws_orig_wiki_page_corpus.pkl', 'rb') as f:
    input_df = pickle.load(f)

In [60]:
count = 0
wiki_object = []
for i in input_df:
    if i['text'] is not None and i['kw'] is not None and i['title'] is not None:
#         wiki_object.append({'text':remove_stopwords(clean_text(i)),'kw':i['kw']})
        wiki_object.append(clean_text_keep_grammar(i))
    count += 1
    print(int(100*count/len(input_df)), end='\r')

In [61]:
count = 0
for i in wiki_object:
        replacement_dict = label_keyword_array(i['kw'], i['text'])
        i['kw'] = replacement_dict
        count += 1
        print(str(count) + '/' + str(len(wiki_object)), end='\r')

KeyboardInterrupt: 

In [ ]:
count = 0
FINAL = []
for i in wiki_object:
    temp_arr = []
    for j in i['kw']:
        if wiki_object[count]['kw'][j] == 'K' or wiki_object[count]['kw'][j] == 'P':
            temp_arr.append(j)
    temp = wiki_object[count]
    temp['kw'] = temp_arr
    FINAL.append(temp)
    count += 1
    print(count, end='\r')

In [ ]:
# with open('/home/jackragless/projects/github/DAIC_glossary_generator/data/wiki_master_dataframe_removed.pkl', 'wb') as f:
#     pickle.dump(FINAL, f)
# with open('/home/jackragless/projects/github/DAIC_glossary_generator/data/wiki_master_dataframe_labelled.pkl', 'wb') as f:
#     pickle.dump(wiki_object, f)
with open('/home/jackragless/projects/github/DAIC_glossary_generator/data/wiki_master_dataframe_chinked_wgrammar.pkl', 'wb') as f:
    pickle.dump(FINAL, f)